# Imports

In [ ]:
from __future__ import nested_scopes
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML('<style>.CodeMirror{font-family: "Courier New";font-size: 12pt;}</style>'))

In [ ]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.ERROR)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import findspark
findspark.init()

In [ ]:
import re
import os
import pandas
pandas.set_option('display.max_rows', None)

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib import colors
from matplotlib import rcParams
rcParams['font.sans-serif'] =  'Courier New'
rcParams['font.family'] = 'Courier New'
rcParams['font.size'] = '12'
%matplotlib inline

from IPython.display import display,HTML
import threading
import collections

from IPython.display import display
from ipywidgets import IntProgress,Layout
import time
import threading
import gzip
import pyspark
import pyspark.sql
from pyspark.sql import SparkSession
from pyspark.sql.types import (StructType, StructField, DateType,
    TimestampType, StringType, LongType, IntegerType, DoubleType,FloatType)
from pyspark.sql.functions import to_date, floor
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import lit
import datetime
import time
from pyspark.storagelevel import StorageLevel
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col
from pyspark.ml import Pipeline
import pandas
import numpy


import re
import math
from functools import reduce
import json


from pyspark.sql.types import *
from pyspark.sql import functions as F
from datetime import date

# start analysis cluster and run

In [ ]:
executors_per_node = 5
nodes=3
cores_per_executor=4
task_per_core=4


cache_size=20
total_size=100000
print('executor per node: {:d}\nparallelism: {:d}\nmemory: {:d}m\noffheap:{:d}m'.format(executors_per_node,nodes*executors_per_node*cores_per_executor*task_per_core,int(math.floor(nodes*total_size/(nodes*executors_per_node)))-1024-int(math.floor(cache_size*1024/(nodes*executors_per_node))),int(math.floor(cache_size*1024/(nodes*executors_per_node)))))

from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
conf = (SparkConf()
    .set("spark.default.parallelism", "{:d}".format(nodes*executors_per_node))
    .set('spark.executor.instances', '{:d}'.format(nodes*executors_per_node))
    .set('spark.sql.files.maxPartitionBytes', '256m')
    .set('spark.app.name', 'pyspark_final')
    .set('spark.rdd.compress', 'False')
    .set('spark.serializer','org.apache.spark.serializer.KryoSerializer')    
    .set('spark.executor.cores','{:d}'.format(cores_per_executor))
    .set('spark.sql.adaptive.coalescePartitions.initialPartitionNum','{:d}'.format(nodes*executors_per_node*cores_per_executor*task_per_core))
    .set('spark.sql.adaptive.enabled',True)
    .set('spark.sql.adaptive.advisoryPartitionSizeInBytes', '256m')
    .set('spark.executor.memory', '{:d}m'.format(int(math.floor(nodes*total_size/(nodes*executors_per_node)))-1024-int(math.floor(cache_size*1024/(nodes*executors_per_node)))))
    .set('spark.task.cpus','1')
    .set('spark.driver.memory','128g')
    .set('spark.sql.inMemoryColumnarStorage.compressed','False')
    .set('spark.sql.inMemoryColumnarStorage.batchSize','100000')
#    .set('spark.memory.storageFraction','0.8')
#    .set('spark.memory.fraction','0.7')
        
    .set('spark.sql.execution.arrow.fallback.enabled','True')
    .set('spark.sql.execution.arrow.enabled','True')
    .set('spark.sql.execution.arrow.maxRecordsPerBatch','100000')
    .set("spark.sql.repl.eagerEval.enabled", True)
        
    .set('spark.memory.offHeap.enabled','True')
    .set('spark.memory.offHeap.size','{:d}m'.format(int(math.floor(cache_size*1024/(nodes*executors_per_node)))))
    .set('spark.executor.memoryOverhead','{:d}m'.format(int(math.floor(cache_size*1024/(nodes*executors_per_node)))+3000))
    .set('spark.sql.join.preferSortMergeJoin','False')
#    .set('spark.executor.extraJavaOptions',
#          '-XX:+UseG1GC -XX:+PrintFlagsFinal -XX:+PrintReferenceGC -verbose:gc -XX:+PrintGCDetails -XX:+PrintGCTimeStamps -XX:InitiatingHeapOccupancyPercent=35')
    .set('spark.executor.extraJavaOptions',
          '-XX:+UseParallelGC -XX:+UseParallelOldGC -verbose:gc -XX:+PrintGCDetails -XX:+PrintGCTimeStamps')
    .set('spark.dynamicAllocation.enabled', 'False')
    .set('spark.shuffle.service.enabled', 'False')
    .set('spark.dynamicAllocation.shuffleTracking.enabled', 'Falze')
    .set("spark.sql.legacy.timeParserPolicy","LEGACY")

        
#    .set("spark.sql.session.timeZone", "PST")
     .set("spark.sql.session.timeZone", "Etc/GMT-8")
#     .set("spark.driver.extraJavaOptions","-Duser.timezone=UTC+8")
    
       )

sc = SparkContext(conf=conf,master='yarn')
sc.setLogLevel("ERROR")
spark = SQLContext(sc)
time.sleep(10)


In [ ]:
%%html
<style>
div.output_stderr {
background: #ffdd;
display: none;
}
</style>

# Sparklog

In [ ]:
%run ./sparklog_monitor.ipynb

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
basedir="BASEDIR_TEMP"
appid="APPID_TEMP"

last_basedir="LAST_BASEDIR"
last_appid="LAST_APPID"

# native, partition table

In [ ]:
app=Application_Run(appid,basedir=basedir)
appals=app.analysis['app']['als']

In [ ]:
appals.get_basic_state()

In [ ]:
appals.get_app_name()

In [ ]:
#shuffle_df, dfx=appals.get_shuffle_stat()

# APP info

In [ ]:
app.generate_trace_view(showemon=False,show_metric=emonmetric,disk_prefix='nvme%n1p1',nic_prefix=["'enp134s0f1'","'eno1'"])

In [ ]:
appals=app.analysis['app']['als']

In [ ]:
appals.get_app_info(disk_prefix='nvme%n1p1',nic_prefix=["'enp134s0f1'","'eno1'"])

In [ ]:
appals.show_critical_path_time_breakdown()